In [1]:
import cython
import numpy as np
%load_ext Cython

In [2]:
%load_ext Cython

In [22]:
def py_fib(n):
    a, b = 0.0, 1.0
    c = np.array([1,1])
    for i in range(n):
        a, b = a + b, a
        c[0]+=1
    return a
%timeit py_fib(1000)

404 µs ± 10.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [23]:
%%cython
cimport numpy as cnp
def cy_fib(int n):
    
    a, b = 0.0, 1.0
    c = cnp.array([1,1])
    for i in range(n):
        a, b = a + b, a
        c[0]+=1
    return a



Error compiling Cython file:
------------------------------------------------------------
...
cimport numpy as cnp
def cy_fib(int n):
    
    a, b = 0.0, 1.0
    c = cnp.array([1,1])
          ^
------------------------------------------------------------

/Users/kaname/.ipython/cython/_cython_magic_acb371067e498791424ae3059742eafd.pyx:5:11: cimported module has no attribute 'array'

Error compiling Cython file:
------------------------------------------------------------
...
cimport numpy as cnp
def cy_fib(int n):
    
    a, b = 0.0, 1.0
    c = cnp.array([1,1])
          ^
------------------------------------------------------------

/Users/kaname/.ipython/cython/_cython_magic_acb371067e498791424ae3059742eafd.pyx:5:11: Compiler crash in AnalyseExpressionsTransform

ModuleNode.body = StatListNode(_cython_magic_acb371067e498791424ae3059742eafd.pyx:1:0)
StatListNode.stats[1] = StatListNode(_cython_magic_acb371067e498791424ae3059742eafd.pyx:2:0)
StatListNode.stats[0] = DefNode(_cython

In [19]:
%timeit cy_fib(1000)

1.18 µs ± 40.6 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [15]:
tt = np.array([1,2,3,4])
np.random.shuffle(tt)
tt

array([1, 4, 2, 3])

In [14]:
t = np.random.rand(90).reshape(-1,3).T
np.random.shuffle(t)
t

array([[0.72144294, 0.1168756 , 0.29321384, 0.6370844 , 0.21458115,
        0.73469876, 0.51731111, 0.5370428 , 0.98375581, 0.34641748,
        0.58331549, 0.05132127, 0.49056708, 0.60382818, 0.54882347,
        0.65790297, 0.26357613, 0.07247005, 0.50174872, 0.63923834,
        0.00414579, 0.47588465, 0.30580955, 0.1775011 , 0.98433375,
        0.84745636, 0.79985993, 0.93420166, 0.3712313 , 0.10830565],
       [0.98375339, 0.62455391, 0.41227799, 0.64412598, 0.19256642,
        0.55633327, 0.0338206 , 0.76536425, 0.65225958, 0.74539392,
        0.62409506, 0.11421422, 0.53642134, 0.73105384, 0.81593137,
        0.49349016, 0.84874369, 0.66660687, 0.75757372, 0.72311282,
        0.2470239 , 0.42792141, 0.68748464, 0.33606094, 0.63651184,
        0.69698557, 0.35802667, 0.85622773, 0.75569755, 0.60615111],
       [0.7487746 , 0.41333302, 0.59162638, 0.34666812, 0.01534741,
        0.61011456, 0.48021391, 0.61095786, 0.81605688, 0.17764075,
        0.33012686, 0.57663459, 0.06409537, 0.

In [3]:
import cython
import numpy as np
%load_ext Cython

In [4]:
def test(n):
    p = np.random.rand(90).reshape(-1,3).T.astype('float32')
    v = np.random.rand(90).reshape(-1,3).T.astype('float32')
    for i in range(n):
        _=getBorderIndex(p,v)
        np.random.shuffle(p)
        np.random.shuffle(v)



In [84]:
def getBorderIndex(p,v):
    Dtype='float32'
    border = np.array([0. , 0.4, 0.7, 1.0], dtype=Dtype)
    margin = 1e-8
    delta  = np.tile(border,(p[2].size,1)).astype(Dtype)\
    -np.tile(p[2],(border.size,1)).astype(Dtype).T
    delta[abs(delta)<margin] = 0
    delta = np.sign(delta).astype('int16')
    ind = np.array(np.where(delta==0))
    ind_shallow = np.array(np.where(ind[1]==0)[0])
    ind_deep = np.array(np.where(ind[1]==(border.size-1))[0])
    if list(ind[0])!= []:
        delta[ind[0],ind[1]] = -np.sign(v[2,ind[0]])
        delta[ind[0,ind_shallow],0] = -1
        delta[ind[0,ind_deep],border.size-1] = 1
    return np.where(delta[:,:-1] != delta[:,1:])[1]

In [78]:
def getBorderIndex(p,v):
    border = np.array([0. , 0.4, 0.7, 1.0], dtype='float32')
    A,B = np.meshgrid(border,p[2])
    delta = A-B
    delta[abs(delta)<1e-8] = 0
    delta[delta<0] = -1
    delta[delta>0] = 1
    delta[:,-1][delta[:,-1]==0] = 1
    delta[:,-1][delta[:,1]==0] = -1
    return delta[:,:-1] - delta[:,1:]

In [85]:
%timeit test(100)

4.26 ms ± 217 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [14]:

a = np.array([1,2,3])
b = np.array([1,1,1,1,1,1])
A,B = np.meshgrid(a,b)


In [90]:
a = []
a[0] =1
a[1] = 2

IndexError: list assignment index out of range

In [17]:
A-B

array([[0, 1, 2],
       [0, 1, 2],
       [0, 1, 2],
       [0, 1, 2],
       [0, 1, 2],
       [0, 1, 2]])

In [100]:
%%cython
cimport numpy as cnp
cnp.import_array()
#import numpy as np
#from numpy import float32 as DTYPE
def getBorderIndex(cnp.ndarray[ndim=3, dtype =float] p,
                   cnp.ndarray[ndim=3, dtype =float] v):
    cdef int p_size,b_size
    cdef cnp.ndarray[ndim = 1, dtype =float] border = cnp.array([0. , 0.4, 0.7, 1.0])
    p_size = p[2].size
    b_size = border.size
    #cdef cnp.ndarray[ndim = 1, dtype =int] 
    index = cnp.zeros(p_size,dtype=int)
    for i in range(p_size):
        for j in range(b_size-1):
            if (p[2][i]>= border[j])and(p[2][i]<= border[j+1]):
                if (p[2][i] == border[j+1])and(v[2][i]>=0):
                    index[i] = j+1
    return index



Error compiling Cython file:
------------------------------------------------------------
...
    cdef int p_size,b_size
    cdef cnp.ndarray[ndim = 1, dtype =float] border = cnp.array([0. , 0.4, 0.7, 1.0])
    p_size = p[2].size
    b_size = border.size
    #cdef cnp.ndarray[ndim = 1, dtype =int] 
    index = cnp.zeros(p_size,dtype=int)
              ^
------------------------------------------------------------

/Users/kaname/.ipython/cython/_cython_magic_61cf59b833c194769b12afd9dd2bf9eb.pyx:12:15: cimported module has no attribute 'zeros'

Error compiling Cython file:
------------------------------------------------------------
...
    cdef int p_size,b_size
    cdef cnp.ndarray[ndim = 1, dtype =float] border = cnp.array([0. , 0.4, 0.7, 1.0])
    p_size = p[2].size
    b_size = border.size
    #cdef cnp.ndarray[ndim = 1, dtype =int] 
    index = cnp.zeros(p_size,dtype=int)
              ^
------------------------------------------------------------

/Users/kaname/.ipython/cython/_c

In [97]:
np.empty(1, dtype=np.int32)

array([897988541], dtype=int32)

In [8]:
%timeit test(100)

4.92 ms ± 346 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [25]:
data = np.zeros((2600,5200))
data[100:200,100:200] = 10

In [49]:
%timeit np.where(data<=10)

181 ms ± 4.06 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [50]:
%timeit data[data<=10]

39.3 ms ± 652 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [27]:
%timeit np.in1d(x.ravel(), goodvalues).reshape(x.shape) 

9.78 ms ± 93.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [30]:
np.unravel_index(np.where(data.ravel()==10),data.shape)

(array([[100, 100, 100, ..., 199, 199, 199]]),
 array([[100, 101, 102, ..., 197, 198, 199]]))

In [31]:
np.where(data==10)

(array([100, 100, 100, ..., 199, 199, 199]),
 array([100, 101, 102, ..., 197, 198, 199]))

In [32]:
np.in1d(data.ravel(), 10).reshape(data.shape) 

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [37]:
data[np.in1d(data.ravel(), 10).reshape(data.shape)]

array([10., 10., 10., ..., 10., 10., 10.])

In [44]:
aa = np.array([[1,2,3,4,5,6,7],
              [1,2,4,4,5,6,7],
              [1,2,10,4,5,6,7]])
issuperset(1,aa)

NameError: name 'issuperset' is not defined

In [46]:
aa[aa<3]=0

In [60]:
aa[:,-1][aa[:,-1]==7] = 8

In [132]:
list(np.random.randint(0,3,10))

[1, 2, 2, 0, 1, 1, 1, 2, 1, 0]

In [249]:
n = 4
x = np.random.randint(0,n-1,10000)
#a = np.zeros((n,len(x))).astype(bool)
a = np.zeros((n,len(x))).astype(bool)
@jit
def test1(n,x,a):
    count = 0
    for i in x:
        a[i,count] = True
        count+=1
    return a


In [250]:
%timeit test1(n,x,a)

6.17 µs ± 160 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [143]:
%timeit test1(n,x)

1.37 ms ± 47.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [151]:
from numba import jit
from numba import prange
def internalDef():
    a = np.random.rand(1000000).sum()
    return np.random.randint(int(a))
@jit(parallel=True)
def test(n):
    a = 0
    for i in prange(n):
        a += internalDef()
test(10)

<ipython-input-151-16550ce819f4>:6: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "test" failed type inference due to: Untyped global name 'internalDef': cannot determine Numba type of <class 'function'>

File "<ipython-input-151-16550ce819f4>", line 10:
def test(n):
    <source elided>
    for i in prange(n):
        a += internalDef()
        ^

  @jit(parallel=True)
<ipython-input-151-16550ce819f4>:6: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "test" failed type inference due to: cannot determine Numba type of <class 'numba.dispatcher.LiftedLoop'>

File "<ipython-input-151-16550ce819f4>", line 9:
def test(n):
    <source elided>
    a = 0
    for i in prange(n):
    ^

  @jit(parallel=True)
/Users/kaname/opt/anaconda3/lib/python3.7/site-packages/numba/object_mode_passes.py:178: NumbaWarning: Function "test" was compiled in object mode without forceobj=True, but has lifte

In [114]:
%timeit test(1000)

9.71 s ± 569 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [105]:
%timeit test(1000)

9.37 s ± 134 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [167]:
a = np.array([[1,2,3,4,5,6],[1,2,3,4,5,6]])
a[0][a[0]<5] and a[1][a[1]<4]

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [284]:
nr,nz = 100,100
dr,dz = 0.02,0.02
Arz = np.zeros((nr,nz),dtype = 'float32')
r = np.array([(i)*dr for i in range(nr+1)])
z = np.array([(i)*dz for i in range(nz+1)])


In [389]:
from numba import njit
def saveFluesnce(self,p,w):
    rr = np.sqrt(p[0]**2+p[1]**2)
    zz = p[2]
    prz = np.array([rr,zz]).astype('float32')
    prz = prz[:,np.argsort(prz[1,:])]
    self.Arz = inputArz(prz,w,self.r,self.z,self.Arz)

@njit('f4[:,:](f4[:,:],f4[:],f8[:],f8[:],f4[:,:])')
def inputArz(prz,w,r,z,Arz):
    count = 0
    nr,nz = r.size,z.size
    for wi in w:
        flag = True
        num_r = 0
        num_z = 0
        val = 0
        for i in r:
            if val==nr-1:
                flag = False
                break
            if (prz[0][count]>=i) and (prz[0][count]<r[val+1]):
                num_r = val
                break
            val+=1
        else:
            continue
        if flag:
            val = 0
            for i in z:
                if val==nz-1:
                    flag = False
                    break
                if (prz[1][count]>=i) and (prz[1][count]<z[val+1]):
                    num_z = val
                    break
                val+=1
            else:
                continue
            if flag:
                Arz[num_r][num_z]+=wi
                count+=1
    return Arz
                

In [390]:
nr,nz = 100,100
dr,dz = 0.01,0.01
Arz = np.zeros((nr,nz),dtype = 'float32')
r = np.array([(i)*dr for i in range(nr+1)]).astype('float32')
z = np.array([(i)*dz for i in range(nz+1)]).astype('float32')


nph = 1000
p = np.random.rand(3,nph).astype('float32')
w = np.random.rand(nph).astype('float32')
rr = np.sqrt(p[0]**2+p[1]**2)
zz = p[2]
prz = np.array([
    rr,zz
]).astype('float32')
prz = prz[:,np.argsort(prz[0,:])]

In [388]:
%timeit inputArz(prz,w,r,z,Arz)

182 µs ± 4.83 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [391]:
%timeit savef(prz,w,r,z,Arz)

212 µs ± 2.09 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [374]:
%timeit savef(prz,w,r,z,Arz)

105 ms ± 1.4 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [353]:
aa = savef(prz,w,r,z,Arz)

3 31 0.908818781375885
4 48 0.9975641369819641
5 33 0.4957249164581299
5 92 0.2216733992099762
6 10 0.024690495803952217
6 13 0.17449010908603668
7 2 0.7709090113639832
7 84 0.0018349656602367759
8 74 0.4621790647506714
8 5 0.4669046998023987
8 79 0.8486335277557373
8 42 0.010183117352426052
8 31 0.4232485890388489
8 71 0.7897064685821533
9 71 0.7502920031547546
10 49 0.7689762711524963
11 85 0.6273386478424072
12 70 0.21328283846378326
12 20 0.51503586769104
12 4 0.6788237690925598
13 1 0.35366469621658325
13 94 0.5075060129165649
13 26 0.33801671862602234
14 19 0.04352061077952385
14 73 0.4047088623046875
14 45 0.847678005695343
15 13 0.23050735890865326
15 68 0.3690536320209503
16 59 0.9541071057319641
16 89 0.661668598651886
16 67 0.5882560014724731
16 92 0.8264049887657166
17 43 0.22468386590480804
17 25 0.07990000396966934
18 75 0.3405619263648987
18 96 0.8130027651786804
18 11 0.4635339081287384
18 86 0.8972289562225342
19 31 0.9839342832565308
19 0 0.8733251690864563
19 15 0.19

92 79 0.1692935675382614
92 91 0.41458046436309814
92 62 0.11282012611627579
92 80 0.03918061777949333
92 75 0.9726340770721436
92 40 0.9135898947715759
92 93 0.637066125869751
92 0 0.40438199043273926
92 80 0.4587595760822296
92 38 0.7459363341331482
92 52 0.16344907879829407
92 96 0.4706682562828064
92 58 0.056470710784196854
92 82 0.2451375126838684
92 17 0.3295302391052246
92 2 0.5402795672416687
92 26 0.1868453323841095
92 23 0.07765749096870422
92 3 0.5693694949150085
93 97 0.9086273908615112
93 47 0.28711646795272827
93 26 0.13856306672096252
93 91 0.44688862562179565
93 20 0.034207534044981
93 14 0.19268931448459625
93 30 0.5871700048446655
93 21 0.6121510863304138
93 95 0.013304638676345348
93 56 0.06341502815485
93 85 0.681921124458313
93 43 0.8678419589996338
93 19 0.9428118467330933
93 4 0.05187535658478737
93 84 0.1808556765317917
93 33 0.38996294140815735
93 34 0.5609729290008545
94 16 0.9996524453163147
94 51 0.6736756563186646
94 42 0.4209746718406677
94 53 0.1971571296

In [356]:
aa[10]

array([0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
       0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
       0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
       0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
       0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
       0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
       0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
       0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
       0.       , 0.7689763, 0.       , 0.       , 0.       , 0.       ,
       0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
       0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
       0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
       0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
       0.       , 0.       , 0.       , 0.       , 

In [348]:
w

array([6.57362342e-01, 1.92361444e-01, 7.14513302e-01, 4.63373989e-01,
       2.68123239e-01, 4.12485272e-01, 7.58065939e-01, 9.19951916e-01,
       5.43394148e-01, 9.36594248e-01, 7.05986738e-01, 3.95480245e-01,
       5.19129694e-01, 4.64719534e-01, 7.99013078e-01, 6.80178583e-01,
       8.68486762e-01, 7.88014591e-01, 6.45538807e-01, 1.93284899e-01,
       6.57376230e-01, 8.00156057e-01, 3.51815522e-01, 6.84918582e-01,
       5.03971457e-01, 5.26508614e-02, 3.93811524e-01, 9.37677503e-01,
       9.17258561e-01, 4.90957916e-01, 4.07019556e-01, 5.49973883e-02,
       5.68105839e-02, 2.85685658e-01, 4.54479545e-01, 1.05371818e-01,
       9.21049416e-01, 8.15739930e-01, 3.87548387e-01, 9.23273802e-01,
       6.92614615e-01, 8.65951955e-01, 1.01338282e-01, 1.18720464e-01,
       2.91307956e-01, 4.44120914e-01, 1.38381392e-01, 2.64705598e-01,
       9.50331748e-01, 1.52252525e-01, 4.64448035e-01, 1.03158519e-01,
       4.80820864e-01, 2.78335601e-01, 7.49606669e-01, 3.42742741e-01,
      

In [228]:
prz[1]

array([9.01786736e-01, 7.09882891e-01, 7.52428538e-02, 2.58198231e-01,
       5.02497227e-01, 7.68504516e-02, 8.64937517e-01, 9.26891339e-02,
       5.31272556e-01, 3.57891312e-01, 3.80045120e-01, 4.06225515e-01,
       9.74512444e-01, 5.30523127e-01, 7.05146878e-01, 7.48920751e-01,
       1.48672952e-01, 2.22364525e-01, 7.39077646e-01, 8.63172732e-01,
       2.64440765e-01, 8.72722314e-02, 8.35804973e-04, 4.09404876e-01,
       2.33807953e-01, 5.70186286e-01, 6.06122168e-01, 7.43069302e-03,
       8.82137709e-01, 1.64417613e-01, 5.84391638e-01, 3.09772396e-01,
       9.47887820e-01, 7.86439548e-01, 5.21185512e-01, 1.56982790e-02,
       2.61766747e-01, 1.39474280e-02, 2.04826528e-01, 3.28754647e-01,
       6.43318786e-01, 8.17113446e-01, 9.22359442e-01, 8.08038678e-01,
       4.06524832e-01, 2.98827882e-01, 3.06010663e-01, 1.03460424e-01,
       7.47050718e-02, 8.37955330e-01, 1.95216906e-01, 4.47102391e-01,
       8.76225921e-01, 3.04508037e-01, 6.97974037e-01, 4.38629919e-01,
      